# PySpark Basique Operations

## 1 Table

Nous allons voir dans un premier temps la manière de selectionner une table, ce qui nous sera utile afin de pouvoir récuperer nos données par le futur.

Pour cela nous allons utiliser la fonction **table()**

Le format est le suivant :

mon_dataframe =  spark.table(**le nom de ma table**)

In [0]:
# En pyspark nous auron donc 
# ma_variable = spark.table('')

# Exécutons maintenant ce code 
mon_dataframe = spark.table('formation.arrivees')

# Affichons notre dataframe afin de nous assurer de la récuperation de notre table
# Pour rappel nous avons deux manières d'afficher nos résultat :

# soit par la fonction display() qui va nous permettre de pouvoir mettre en forme nos résultats grâce aux options présentes en dessous 
display(mon_dataframe)

# soit par la fonction show() qui nous permet également d'afficher notre dataframe. On peut lui donner des paramètres lui indiquant la manière d'afficher le dataframe
mon_dataframe.show()

>Nous pouvons constater que nous récupérons bien notre dataframe !  
>Nous pouvons également limiter le nombre de résultat renvoyé grâce à la fonction limit()

In [0]:
# Exemple avec la fonction display
mon_dataframe = mon_dataframe.limit(5)
display(mon_dataframe)

# Exemple avec la fonction show()
mon_dataframe.show(5)

## 2 Select

Maintenant que nous avons récupéré notre table, nous allons selectionner les colonnes qui nous intéressent grâce au **select()**

Le format est le suivant :

mon_dataframe =  spark.table(**le nom de ma table**).select(**les noms de mes colonnes**)

In [0]:
# nous rajoutons aprés avoir récupéré notre table la selection de nos colonnes 
mon_dataframe =  spark.table('formation.arrivees').select("numero_caf","date_jour")
display(mon_dataframe)

# nous pouvons constater que le retour nous affiche bel et bien que deux colonnes

In [0]:
%sql

-- Si nous devions le traduire en SQL simple cela serait l'équivalent de :
SELECT numero_caf,date_jour from formation.arrivees;


### 2.1 Selection de colonnes

Nous avons vu la selection de colonnes grâce au **select()**  
Il est également possible de saisir les colonnes grâce à **col()**

In [0]:
import pyspark.sql.functions as F

# nous rajoutons aprés avoir récupéré notre table la selection de nos colonnes 
mon_dataframe =  spark.table('formation.arrivees').select(F.col('numero_caf'),F.col('date_jour'))

display(mon_dataframe)

# Exemple de la documentation officiel de Pyspark
# def col(col: str) -> Column:
#    """
#    Returns a :class:`~pyspark.sql.Column` based on the given column name.'
#    Examples
#    --------
#    >>> col('x')
#    Column<'x'>
#    >>> column('x')
#    Column<'x'>
#    """
#    return _invoke_function("col", col)

# On peut également se servir de col() afin de n'utiliser que cette colonne 
my_col = F.col("numero_caf")

# On affiche que notre colonne récupéré ci-dessus à partir de notre dataframe en limitant notre retour à 5 résultat
mon_dataframe.select(my_col).show(5)


### 2.2 Ajout de colonnes

Nous avons la possibilité également d'ajouter des colonnes grâce à la fonction **withColumn()**

Mais celle-ci ne permet pas que cela, elle peut aussi permettre :
- Le changement de type d'une colonne
- L'actualisation des valeurs contenues dans une colonne
- La création d'une colonne à partir d'une existante
- Le renommage d'une colonne

#### Ajout de colonne Exemple

Le format est le suivant pour l'ajout d'une colonne :

`mon_dataframe = spark.table('ma_table').select(mes_colonnes)`

`mon_dataframe = mon_dataframe.withColumn("nom_de_ma_nouvelle_colonne",condition)`

##### Exercice

Ajouter une colonne nous permettant de savoir si **date_jour** est supérieur ou non à 2021

In [0]:
# Nous récupérons notre dataframe
mon_dataframe = spark.table('formation.arrivees').select("numero_caf","date_jour","service")

#Exemple avec l'ajout d'une colonne nous indiquant si la date est supérieur à 2021
mon_dataframe.withColumn("date_2021", mon_dataframe["date_jour"]>"2021").show(5)

# Autre exemple de réponse 
df = spark.table('formation.arrivees').select("numero_caf" , "date_jour","service").withColumn("nouvelle colonne" , F.when( F.col("date_jour") > "2021-01-01", ).otherwise(0) ).show()

# Autre exemple de réponse
df = spark.table('formation.arrivees').select("numero_caf","date_jour","service").withColumn('2021', F.col("date_jour").between('2021-01-01', '2021-12-31')).display()

#### Exemple Concret 

Cette fois-ci nous allons prendre un exemple concret avec l'ajout de colonnes afin d'établir des chiffres sur les pièces fournits.

##### Exercice

Sur la table "formation.arrivees" ajouter des colonnes nous permettant de savoir si la colonne "nombre_piece_masse" est 
  - inferieur a "nombre_piece_total"
  - inferieur ou égal a "nombre_piece_total"
  - supérieur a "nombre_piece_total"
  - supérieur ou égal a "nombre_piece_total"
  - égal a "nombre_piece_total"

In [0]:
mon_dataframe = spark.table('formation.arrivees').select("numero_caf","date_jour","nombre_courrier","nombre_piece_masse","nombre_piece_total").limit(10)
df = mon_dataframe.withColumn("lt",mon_dataframe["nombre_piece_masse"]<mon_dataframe["nombre_piece_total"])\
       .withColumn("gt",mon_dataframe["nombre_piece_masse"]>mon_dataframe["nombre_piece_total"])\
       .withColumn("lte",mon_dataframe["nombre_piece_masse"]<=mon_dataframe["nombre_piece_total"])\
       .withColumn("gte",mon_dataframe["nombre_piece_masse"]>=mon_dataframe["nombre_piece_total"])\
       .withColumn("eq",mon_dataframe["nombre_piece_masse"]==mon_dataframe["nombre_piece_total"])

display(df)

> Nous pouvons transformer le retour de réponse (en replaçant les True/False par 0 ou 1) afin de faciliter la récupération et le compte de celle-ci grâce au **when** que nous allons voir juste aprés

In [0]:
df = df.withColumn("lt" , F.when(df.nombre_piece_masse < df.nombre_piece_total, 1).otherwise(0) )\
        .withColumn("lte" , F.when(df.nombre_piece_masse <= df.nombre_piece_total, 1).otherwise(0) )\
        .withColumn("gt" , F.when(df.nombre_piece_masse > df.nombre_piece_total, 1).otherwise(0) )\
        .withColumn("gte" , F.when(df.nombre_piece_masse >= df.nombre_piece_total, 1).otherwise(0) )\
        .withColumn("eq" , F.when(df.nombre_piece_masse >= df.nombre_piece_total, 1).otherwise(0) )

#### Ajout de colonne avec lit

Nous avons la possibilité également lors de l'ajout de colonne de définir une valeur constante ou une valeur littérale grâce à la fonction **lit()** de Pyspark

In [0]:
from pyspark.sql.functions import lit
# Nous récupérons notre dataframe
mon_dataframe = spark.table('formation.arrivees').select("numero_caf","date_jour","service")

mon_dataframe.show(5)

#Exemple avec l'ajout d'une colonne avec la fonction lit()
mon_dataframe.withColumn("colonne_constante", lit("FR") ).show(5)

#### Ajout de colonne avec conditions multiples et lit

On peut ajouter une colonne avec des conditions multiples, dans le premier exemple nous venons insérer une colonne qui affichera si "date_jour" est supérieur à 2021  
Nous pouvons enchainer les conditions multiples grâce au **when()**

In [0]:
from pyspark.sql.functions import when, lit
# Nous récupérons notre dataframe
mon_dataframe = spark.table('formation.arrivees').select("numero_caf","date_jour","service")

mon_dataframe.show(5)

#Exemple avec l'ajout d'une colonne avec la fonction lit()
df = mon_dataframe.withColumn("colonne_conditions", \
                         when((mon_dataframe.date_jour < "2019"), lit("past 2019"))
                        .when((mon_dataframe.date_jour >= "2019") & (mon_dataframe.date_jour < "2020"),lit("past 2020"))
                        .otherwise(lit("PRESENT")))

display(df)

### 2.3 Création de colonne avec Pivot

La fonction **pivot()** de Pyspark permet la création de colonnes de manière différente.  
En effet, ces colonnes vont être le fruit de la division ou la transposition d'une autre colonne.  

On pourrait simplifier cela en disant que l'on va tranposer les donnée d'une colonne en plusieurs.  

La notion importante de la fonction d'agrégation **pivot()** est que les données une fois transposées auront subis le **distinct**

#### Représentation concrête

```
+-------------------+--------+
|COLONNE A PIVOT    | VALEUR |
+-------------------+--------+
|future colonne1    | val1   |
|future colonne2    | val2   |
|future colonne3    | val3   |
|future colonne4    | val4   |
+-------------------+--------+
```

Ma table aprés mon opération de **pivot()** deviendra cela dans sa forme la plus simple :

```
+----------------+------------------+-----------------+-----------------+
|future colonne1 | future colonne2  | future colonne3 | future colonne4 |
+----------------+------------------+-----------------+-----------------+
|      val1      |      val2        |      val3       |      val4       |
+----------------+------------------+-----------------+-----------------+
```

#### Exemple avec une table fictive simple

In [0]:
# Création de données fictives
data = [("Banane",50,"USA"),("Orange",40,"ESP"),("Avocat",200,"USA"),("Melon",15,"ESP"),("Citron",110,"ITA"),("Citron vert",75,"ITA"),]

# Création de nos appelations de colonnes
columns= ["Fruit","Quantite","Provenance"]

# Création de notre dataframe
mon_dataframe = spark.createDataFrame(data = data, schema = columns)

# On peut afficher son schéma si on veut
# mon_dataframe.printSchema()

# Affichage de notre dataframe
display(mon_dataframe)



# Mise en pratique de notre fonction d'agrégation pivot()


# On va donc ici grouper nos rows par Fruit
# Transposer notre colonne "Provenance" en plusieurs
# Et faire la somme de "Quantite" par "Provenance" pour chaque "Fruit"
mon_dataframe = mon_dataframe.groupBy("Fruit").pivot("Provenance").sum("Quantite").show(10)

# On va donc ici grouper nos rows par Provenance
# Transposer notre colonne "Fruit" en plusieurs
# Et faire la somme de "Quantite" pour chaque "Fruit" pour chaque "Provenance"
mon_dataframe = mon_dataframe.groupBy("Provenance").pivot("Fruit").sum("Quantite").show(10)

#### Exemple concret avec notre table formation.arrivees

In [0]:
# Nous récupérons notre dataframe
mon_dataframe = spark.table('formation.arrivees').select("numero_caf","date_jour","service","nombre_courrier","nombre_piece_total")

# Nous allons pour exemple prendre un pivot fait avec la colonne "service"
# La traduction de notre requete serait :
    # Groupe mes rows par numéro de caf
    # transpose ma colonne service en colonnes séparées (un identifiant de service devient une colonne)
    # fait le total de courrier par service pour chaque numero_caf
mon_dataframe = mon_dataframe.groupBy("numero_caf").pivot("service").sum("nombre_courrier")

display(mon_dataframe)

#### Ressources avec Pivot

> l'implémentation de **pivot()** à été amélioré au fur et à mesure des versions de Pyspark, mais cela reste une fonction qui consomme des ressources.

## 3 Filtre

La méthode **filter()** va nous permettre une fois notre table,nos colonnes selectionnées d'appliquer un filtre sur les résultats retournés.  

> Plusieurs filtres peuvent être appliqués en même temps  
> La méthode **filter()** correspond à la clause **WHERE** en SQL

Le format est le suivant :

mon_dataframe =  spark.table('formation.arrivees').select("numero_caf","date_jour").filter(**ma conditon**)

In [0]:
# Nous appliquons le filtrage en deux fois afin dans un premier de récuperer notre dataframe puis de le filtrer
mon_dataframe = spark.table('formation.arrivees').select("numero_caf","date_jour","service")

# Nous filtrons en venant appliquer la condition à notre colonne
mon_dataframe = mon_dataframe.filter(mon_dataframe.numero_caf != "001")

# La méthode filter permet d'écrire les conditions sous plusieurs formes
# Nous reprendrons ci-dessous le format de condition propre au SQL
mon_dataframe = mon_dataframe.filter("numero_caf != 001")

# Cela sous-entend que nous pouvons reprendre tous les opérateurs de conditions SQL 

# Affichage de notre dataframe
display(mon_dataframe)


### 3.1 Exemple de filtrages différents

#### Filtre multiple 

> Nous allons pour ce faire utiliser les opérateurs ( "&", "|" ) qui correspondent au ( "AND", "OR" ) en SQL

In [0]:
mon_dataframe = spark.table('formation.arrivees').select("numero_caf","date_jour","service")

# Exemple avec des conditions multiples et l'affichage avec la méthode show()
mon_dataframe = mon_dataframe.filter((mon_dataframe.numero_caf != "001" ) & (mon_dataframe.service == "AS") ).show()

display(mon_dataframe)


#### Filtre basé sur une liste 

> Nous allons pour ce faire utilisé le mot clé **isin()**  
> Nous lui passerons notre liste en paramètre

In [0]:
# Nous pouvons également appliquer des filtres ayant une liste comme entrée 
# Imaginons que dans notre cas nous voulons faire un filtrage sur plusieurs services 

# Nous définissons notre liste
liste_service=["AS","PF","AC"]

mon_dataframe = spark.table('formation.arrivees').select("numero_caf","date_jour","service")

# Nous appliquons notre filtrage sur la colonne qui sera verifié en fonction de notre liste
mon_dataframe = mon_dataframe.filter(mon_dataframe.service.isin(liste_service))

display(mon_dataframe)

#### Filtre basé sur le début et la fin d'un champ

> Nous utiliserons pour ce faire les mots clés **startswith()** et **endswith()**

In [0]:
# Nous appliquons notre filtrage en fonction de la première lettre de mon champ, ici un A
# On affiche grâce à la méthode show() et on limite les nombres de résultats retournés à 5
mon_dataframe = spark.table('formation.arrivees').select("numero_caf","date_jour","service")
mon_dataframe = mon_dataframe.filter(mon_dataframe.service.startswith("A")).show(5)

# Nous appliquons notre filtrage en fonction de la dernière lettre de mon champ, ici un U
# On affiche grâce à la méthode show() et on limite les nombres de résultats retournés à 5
mon_dataframe = spark.table('formation.arrivees').select("numero_caf","date_jour","service")
mon_dataframe = mon_dataframe.filter(mon_dataframe.service.endswith("U")).show(5)



#### Filtre et suite

D'autre filtres existent également voici une liste non-exhaustive :  

- filtrage pour vérifier la contenance grâce au contains()

- filtrage reprenant le **like** et le **rlike** du SQL 

  - like permet de match les valeurs avec le pattern fournit (attention toutefois like est **case sensitive** à la différence de **rlike** (regex like))

## 4 Verifier nos rows

Parfois malgré le filtrage possible, nous voulons quand même pouvoir vérifier la nullité d'un row.

Pour cela nous avons la fonction **isnull()**

Nous avons plusieurs manières d'intégrer **isnull()**

- au travers d'un **select** afin de tester la condition s'il le row est null ou pas
- au travers des **filtres** afin de retourner les rows null ou pas

In [0]:
# Première méthode au travers d'un SELECT

# Nous importons nos fonctions SQL
import pyspark.sql.functions as F

# Nous changeons de table afin d'être sur d'avoir des champs "null"
# Nous appliquons directement le isnull dans notre select afin de récuperer un row qui nous renverra la valeur booléenne de la fonction isnull()
# Cela revient pour chaque row à tester si celui-ci est null ou pas 
mon_dataframe = spark.table('bgescaf.gssdparr').select("NUMCAF","DTJOUR",F.isnull(F.col("NPIEMA")),F.isnull(F.col("PPIEMA")))

display(mon_dataframe)

# On peut également écrire cela de cette façon :
# On récupère notre dataframe au travers d'un SELECT
mon_dataframe2 = spark.table('bgescaf.gssdparr').select("NUMCAF","DTJOUR","NPIEMA","PPIEMA")

# Puis on SELECT de nouveau mais cette fois-ci en appliquant notre méthode isNull()
mon_dataframe2 = mon_dataframe2.select("NUMCAF","DTJOUR",mon_dataframe2.NPIEMA.isNull(),mon_dataframe2.PPIEMA.isNull())

display(mon_dataframe2)

In [0]:
# Deuxième méthode au travers d'un filtre

# Nous importons nos fonctions SQL *
import pyspark.sql.functions as F

# Nous changeons de table afin d'être sur d'avoir des champs "null"
mon_dataframe = spark.table('bgescaf.gssdparr').select("NUMCAF","DTJOUR","NPIEMA","PPIEMA")

# Nous filtrons sur la valeur null
mon_dataframe = mon_dataframe.filter(mon_dataframe.NPIEMA.isNull())

# Autre manière d'écrire le filtre :
# mon_dataframe = mon_dataframe.filter("NPIEMA IS NOT NULL")
# mon_dataframe = mon_dataframe.filter(F.col("NPIEMA").isNull())

display(mon_dataframe)

## 5 Doublons de rows

Nos tables peuvent contenir des rows en double.  

Afin de palier cela et d'afficher nos rows sans doublons nous allons avoir recours à deux fonctions plus ou moins similaires.

- **distinct()**
- **dropDuplicate()**

### 5.1 Suppression des doublons avec distinct()

In [0]:
# Nous récupérons notre dataframe
mon_dataframe = spark.table('formation.arrivees').select("numero_caf")
print("Nombre de rows : " + str(mon_dataframe.count()))

display(mon_dataframe)

# Nous récupérons notre dataframe mais cette fois-ci sans les doublons grâce au distinct()
mon_dataframe2 = spark.table('formation.arrivees').select("numero_caf").distinct()
print("Nombre de rows sans les doublons : " + str(mon_dataframe2.count()))

display(mon_dataframe2)


### 5.2 Suppression des doublons avec dropDuplicates()

In [0]:
# Nous récupérons notre dataframe
mon_dataframe = spark.table('formation.arrivees').select("numero_caf","service")
print("Nombre de rows : " + str(mon_dataframe.count()))

display(mon_dataframe.orderBy("numero_caf"))

# Nous récupérons notre dataframe mais cette fois-ci sans les doublons grâce au distinct()
mon_dataframe2 = spark.table('formation.arrivees').select("numero_caf","service")
mon_dataframe2 = mon_dataframe2.dropDuplicates(["numero_caf","service"])
print("Nombre de rows sans les doublons : " + str(mon_dataframe2.count()))

display(mon_dataframe2.orderBy("numero_caf"))

### 5.3 Différence entre les deux

La grande différence entre ces deux méthodes, réside principalement dans l'utilisation que vous allez en faire.  

En effet :

- **distinct()** permet de supprimer les doublons du dataframe en prenant en compte l'ensemble des colonnes de la table.

A contrario 

- **dropDuplicates()** permet de supprimers les doublons du dataframe en prenant en paramètre les colonnes que vous choisirez

## 6 Ordonner nos rows


Aprés avoir vérifier nos rows, nous allons les ordonners.

Pour cela nous allons principalement utiliser deux fonctions :

- sort()
- orderBy()

Toutes deux similaires à leurs homonymes en SQL

#### 6.1 Sort

La fonction **sort()** permet d'ordonner les données en fonction de la ou les colonnes qui lui sont donnée en paramètres.  
Elle prend également en paramètre un booléen afin de savoir si elle ordonne de manière croissante ou pas.

> Par défaut la manière croissante sera toujours choisie

In [0]:
import pyspark.sql.functions as F

# Nous récupérons notre dataframe (cette fois-ci avec des colonnes contenant des quantitées)
mon_dataframe = spark.table('formation.arrivees').select("numero_caf","date_jour","nombre_courrier","nombre_piece_masse","service")

mon_dataframe.show(5)

# Plusieurs manières d'écrire le sort sont encore une fois disponible
# Dans l'exemple ci-dessous nous appliquons notre "sort" sur la colonne "nombre_courrier" de manière décroissante
mon_dataframe.sort(["nombre_courrier"],ascending=[False]).show(10)

# A l'inverse nous aurions pu également trier par ordre croissant en passant notre booleén à "True"
mon_dataframe.sort(["numero_caf"],ascending=[True]).show(10)



# Nous pourrions également l'écrire de ces manière :

# 1
mon_dataframe.sort(mon_dataframe.numero_caf.asc()).show(5)
# Ou a l'inverse 
mon_dataframe.sort(mon_dataframe.numero_caf.desc()).show(5)


# 2 
mon_dataframe.sort(F.col("numero_caf").asc()).show(4)
# Ou a l'inverse 
mon_dataframe.sort(F.col("numero_caf").desc()).show(4)


# Sort plusieurs colonnes
mon_dataframe.sort(["nombre_courrier","nombre_piece_masse"],ascending=[False]).show(20)
mon_dataframe.sort("nombre_courrier","nombre_piece_masse",ascending=[False]).show(20)



#### 6.2 Order By

La fonction **orderBy()** permet d'ordonner les données en fonction de la ou les colonnes qui lui sont donnée en paramètres.  
Elle peut prendre également en paramètre un booléen afin de savoir si elle ordonne de manière croissante ou pas.

> Par défaut la manière croissante sera toujours choisie

In [0]:
import pyspark.sql.functions as F

# Nous récupérons notre dataframe (cette fois-ci avec des colonnes contenant des quantitées)
mon_dataframe = spark.table('formation.arrivees').select("numero_caf","date_jour","nombre_courrier","nombre_piece_masse","service")

mon_dataframe.show(5)

# Plusieurs manières d'écrire le sort sont encore une fois disponible
# Dans l'exemple ci-dessous nous appliquons notre "sort" sur la colonne "nombre_courrier" de manière décroissante
mon_dataframe.orderBy(["nombre_courrier"],ascending=[False]).show(10)

# A l'inverse nous aurions pu également trier par ordre croissant en passant notre booleén à "True"
mon_dataframe.orderBy(["numero_caf"],ascending=[True]).show(10)



# Nous pourrions également l'écrire de ces manière :

# 1
mon_dataframe.orderBy(mon_dataframe.numero_caf.asc()).show(5)
# Ou a l'inverse 
mon_dataframe.orderBy(mon_dataframe.numero_caf.desc()).show(5)


# 2 
mon_dataframe.orderBy(F.col("numero_caf").asc()).show(4)
# Ou a l'inverse 
mon_dataframe.orderBy(F.col("numero_caf").desc()).show(4)


# Sort plusieurs colonnes
mon_dataframe.orderBy(["nombre_courrier","nombre_piece_masse"],ascending=[False]).show(20)
mon_dataframe.orderBy("nombre_courrier","nombre_piece_masse",ascending=[False]).show(20)


#### 6.3 Similitudes et différences

Comme nous avons pu le constater ces deux fonctions :

* ces deux fonctions s'écrivent de la manière
* ces deux fonctions produisent des résultats similaires


Malgré tout cela, il existe quand même des différences :


* En réalité la fonction **orderBy()** est un alias de sort() en Python comme en témoigne ici le code source de Pyspark

```python
def sort(self, *cols, **kwargs):
        """Returns a new :class:`DataFrame` sorted by the specified column(s).
        .. versionadded:: 1.3.0
        Parameters
        ----------
        cols : str, list, or :class:`Column`, optional
             list of :class:`Column` or column names to sort by.
        Other Parameters
        ----------------
        ascending : bool or list, optional
            boolean or list of boolean (default ``True``).
            Sort ascending vs. descending. Specify list for multiple sort orders.
            If a list is specified, length of the list must equal length of the `cols`.
        Examples
        --------
        >>> df.sort(df.age.desc()).collect()
        [Row(age=5, name='Bob'), Row(age=2, name='Alice')]
        >>> df.sort("age", ascending=False).collect()
        [Row(age=5, name='Bob'), Row(age=2, name='Alice')]
        >>> df.orderBy(df.age.desc()).collect()
        [Row(age=5, name='Bob'), Row(age=2, name='Alice')]
        >>> from pyspark.sql.functions import *
        >>> df.sort(asc("age")).collect()
        [Row(age=2, name='Alice'), Row(age=5, name='Bob')]
        >>> df.orderBy(desc("age"), "name").collect()
        [Row(age=5, name='Bob'), Row(age=2, name='Alice')]
        >>> df.orderBy(["age", "name"], ascending=[0, 1]).collect()
        [Row(age=5, name='Bob'), Row(age=2, name='Alice')]
        """
        jdf = self._jdf.sort(self._sort_cols(cols, kwargs))
        return DataFrame(jdf, self.sql_ctx)

    orderBy = sort
```

En réalité la différence se joue au niveau de SPARKSQL comme en témoigne la documentation officielle :

  * La fonction **sort()** va permettre de renvoyer les lignes de résultat triées dans chaque partition dans l'ordre spécifié par l'utilisateur. Lorsqu'il y a plusieurs partitions, SORT BY peut renvoyer un résultat partiellement ordonné.

A la différence de la fonction orderBy() :

  * la fonction **orderBy()** va permettre de renvoyer les lignes de résultat de manière triée dans l'ordre spécifié par l'utilisateur. Contrairement à la clause SORT BY, cette clause garantit un ordre total dans la sortie.

## 7 Grouper nos rows

La fonction **groupBy()** est utilisé pour collecter les données identiques et les rassemblers en groupe.  
Elle permet par la suite d'exécuter des fonctions d'agrégation sur les données groupées

En voici une liste non-exhaustive :
- count() : cela renvoie le nombre de row pour chaque groupe
- mean() : cela renvoie la valeur moyenne pour chaque groupe
- min() : cela renvoie la valeur minimum pour chaque groupe
- max() : cela renvoie la valeur maximum pour chaque groupe
- sum() : cela additionne les différentes valeurs et retourne le résultat pour chaque groupe
- avg() : cela renvoie la valeur moyenne pour chaque groupe

In [0]:
# Nous récupérons notre dataframe (cette fois-ci avec des colonnes contenant des quantitées)
mon_dataframe = spark.table('formation.arrivees').select("numero_caf","date_jour","nombre_courrier","nombre_piece_masse","service")

# Exemple avec un groupBy() avec un count()
# Nous comptons ici le nombre de row pour chaque numero_caf et nous ordonnons le tout de manière croissante sur le numéro de caf
mon_dataframe.groupBy("numero_caf").count().orderBy("numero_caf").show(5)

# Exemple avec un groupBy() avec un mean()
# Nous regroupons pour chaque numero de caf le nombre moyen de courrier et nous ordonnons le tout de manière croissante sur le numéro de caf
mon_dataframe.groupBy("numero_caf").mean("nombre_courrier").orderBy("numero_caf").show(5)

# Exemple avec un groupBy() avec un min()
# Nous regroupons pour chaque numero de caf le nombre minimum de courrier et nous ordonnons le tout de manière croissante sur le numéro de caf
mon_dataframe.groupBy("numero_caf").min("nombre_courrier").orderBy("numero_caf").show(5)

# Exemple avec un groupBy() avec un max()
# Nous regroupons pour chaque numero de caf le nombre maximum de courrier et nous ordonnons le tout de manière croissante sur le numéro de caf
mon_dataframe.groupBy("numero_caf").max("nombre_courrier").orderBy("numero_caf").show(5)

# Exemple avec un groupBy() avec un sum()
# Nous regroupons pour chaque numero de caf le nombre total (ou la somme total) de courrier et nous ordonnons le tout de manière croissante sur le numéro de caf
mon_dataframe.groupBy("numero_caf").sum("nombre_courrier").orderBy("numero_caf").show(5)


# Autre manière de procéder

# reponse Guillaume
mon_dataframe = mon_dataframe.groupBy("numero_caf").agg(F.max("nombre_courrier")).show()

# reponse Remi
plouf = spark.table("formation.arrivees").groupBy("numero_caf").avg("nombre_courrier").display()

# reponse de Eline
somme_nb_courrier=df.groupBy("numero_caf","service").agg(F.sum("nombre_courrier").alias("total_nombre_courrier"))

## 8 Joindre nos rows

Définition simple :  
La jointure permet d'associer les rows de deux tables

Rappel sur les différents types de jointures :

![jointure](<https://miro.medium.com/max/1400/1*Lb3WTGX-N6HunApw-jT16g.png>)

### 8.1 Jointure simple ou inner join

En SQL comme en Pyspark la jointure simple ou **inner join** peut se faire uniquement lorsque la condition est "vrai" dans les deux tables.

C'est à dire de manière très simple, que l'égalité entre deux colonnes doit être toujours "**vrai**"

> C'est cette condition que l'on va exprimer lors de notre jointure afin de lier nos tables.

In [0]:
# Récupérons notre premier table A
mon_dataframe_A = spark.table("formation.arrivees")
display(mon_dataframe_A.limit(1))

# Récupérons notre deuxième table B
mon_dataframe_B = spark.table("formation.stockees")
display(mon_dataframe_B.limit(1))


Lorsque l'on regarde nos deux table qu'elle sont les champs que l'on retrouve de part et d'autre ?  
On peut constater que nos deux tables sont pratiquement similaire :
- seul le champ delai est rajouté dans la table "formation.stockees"

Nous avons donc de quoi réaliser notre condition

In [0]:
# Récupérons notre premier table A
mon_dataframe_A = spark.table("formation.arrivees")
#display(mon_dataframe_A.limit(1))

# Récupérons notre deuxième table B
mon_dataframe_B = spark.table("formation.stockees")
#display(mon_dataframe_B.limit(1))


# Procédons à notre jointure simple 
# Nous avons joins notre table formation.stockees à notre table formation.arrivees, en lui indiquant notre condition
# Ici nous allons basé notre jointure sur l'équivalence entre nos champs de la colonne numero_caf
mon_dataframe_C = mon_dataframe_A.join(mon_dataframe_B,mon_dataframe_A.numero_caf == mon_dataframe_B.numero_caf,"inner").drop(mon_dataframe.numero_caf)
display(mon_dataframe_C)

# Nous procédons également à une jointure simple mais cette fois-ci avec une double condition
# L'égalité entre les champs numero_caf et date_jour doit être "vrai"
mon_dataframe_D = mon_dataframe_A.join(mon_dataframe_B, on=((mon_dataframe_A["numero_caf"]==mon_dataframe_B["numero_caf"]) & (mon_dataframe_A["date_jour"]==mon_dataframe_B["date_jour"])), how="inner")

# Affichage jointure simple
display(mon_dataframe_C)

# Affichage jointure conditions multiples
display(mon_dataframe_D)

# Permettant de supprimer la colonne de jointure dans le résultat, manière plus lisible d'écrire
mon_dataframe_E = mon_dataframe_A.join(mon_dataframe_B, on="numero_caf",how="inner")
# On peut également le faire avec des colonnes multiples
mon_dataframe_F = mon_dataframe_A.join(mon_dataframe_B, on=["numero_caf","date_jour"],how="inner")

## Pour supprimer le doublon de notre colonne "numero_caf" on peut utiliser .drop(mon_dataframe.numero_caf).show()

Nous pouvons constater que notre jointure à recouper les résultats de nos deux tables.

> Par défaut le "**inner**" présent dans notre requête n'est pas obligatoire, il est le type de jointure par défaut si nous n'en précisons pas.


> **Important** : Tous les rows présents dans nos tables pour lesquels la conditon n'est pas vérifié sont "**drop**".
> C'est à dire qu'il n'apparaitront pas dans les résultats de notre requête !

### 8.2 Jointure croisé ou Cross join 

Le jointure cross permet d'effectuer un produit cartesien entre les deux tables.

Retourne les combinaisons de chaques paires des deux table.

S'il y a 100 lignes dans les tables, il y aura alors 10 000 lignes en résultat.

Vous trouverez une représentation graphique juste en dessous 

![cross](<https://storage.googleapis.com/hackersandslackers-cdn/2019/07/CROSSJOIN.jpg>)

In [0]:
# Nous récuperons nos dataframe
mon_dataframe_A = spark.table("formation.arrivees")
mon_dataframe_B = spark.table("formation.arrivees")

# Affichage du nombre de rows -> 409317
print(mon_dataframe_A.count())

# Nous appliquons la jointure 
mon_dataframe_C = mon_dataframe_A.join(mon_dataframe_B , how="cross")

# Affichage du nombre de rows apres cross jointure ->  167540406489 (cela correspond bien à 409317x409317)
print(mon_dataframe_C.count())

# Nous pouvons également l'écrire de cette manière
mon_dataframe_C = mon_dataframe_A.crossJoin(mon_dataframe_B)

display(mon_dataframe_C)

### 8.3 Jointure Full,Full Outer, Outer

De manière trés simple les jointures **full** nous permettent de joindre nos rows, que l'égalité soit vrai ou pas.

> Il remplacera les rows ou l'égalité n'est pas vérifié par des rows "**null**"

Le format est la suivante :
* Full
  - `table1.join(table2, on=table1["nombre1"]==table2["nombre2"],how="full").display()`
* Full Outer
  - `table1.join(table2, on=table1["nombre1"]==table2["nombre2"],how="full_outer").display()`
* Outer
  - `table1.join(table2, on=table1["nombre1"]==table2["nombre2"],how="outer").display()`

Exemple de jointure **FULL**

In [0]:
# Nous récuperons nos dataframe
mon_dataframe_A = spark.table("formation.arrivees")
mon_dataframe_B = spark.table("formation.arrivees")

# Affichage du nombre de rows -> 409317
print(mon_dataframe_A.count())

# Nous appliquons la jointure 
mon_dataframe_C = mon_dataframe_A.join(mon_dataframe_B ,on=((mon_dataframe_A["numero_caf"]==mon_dataframe_B["numero_caf"]) & (mon_dataframe_A["date_jour"]==mon_dataframe_B["date_jour"])), how="full")

# Affichage du nombre de rows -> 1730575
print(mon_dataframe_C.count())

# Affichage de notre table
display(mon_dataframe_C)

Exemple de jointure **FULL OUTER**

In [0]:
# Nous récuperons nos dataframe
mon_dataframe_A = spark.table("formation.arrivees")
mon_dataframe_B = spark.table("formation.arrivees")

# Affichage du nombre de rows -> 409317
print(mon_dataframe_A.count())

# Nous appliquons la jointure 
mon_dataframe_C = mon_dataframe_A.join(mon_dataframe_B ,on=((mon_dataframe_A["numero_caf"]==mon_dataframe_B["numero_caf"]) & (mon_dataframe_A["date_jour"]==mon_dataframe_B["date_jour"])), how="full_outer")

# Affichage du nombre de rows -> 1730575
print(mon_dataframe_C.count())

# Affichage de notre table
display(mon_dataframe_C)

Exemple de jointure **OUTER**

In [0]:
# Nous récuperons nos dataframe
mon_dataframe_A = spark.table("formation.arrivees")
mon_dataframe_B = spark.table("formation.arrivees")

# Affichage du nombre de rows -> 409317
print(mon_dataframe_A.count())

# Nous appliquons la jointure 
mon_dataframe_C = mon_dataframe_A.join(mon_dataframe_B ,on=((mon_dataframe_A["numero_caf"]==mon_dataframe_B["numero_caf"]) & (mon_dataframe_A["date_jour"]==mon_dataframe_B["date_jour"])), how="outer")

# Affichage du nombre de rows -> 1730575
print(mon_dataframe_C.count())

# Affichage de notre table
display(mon_dataframe_C)

### 8.4 Jointure gauche ou droite 

Les jointures `LEFT` et `RIGHT` permettent de lister tous les résultats de la table de gauche ou de droite avec notre table de référence.

> Ces jointures s'appliquent même s'il n'y a pas d'équivalence entre les champs de nos tables
> Auquel cas les rows seront à **NULL**

Le format est la suivante :
* LEFT (ou appelé communément en SQL `LEFT OUTER JOIN`)
  - `table1.join(table2, on=table1["nombre1"]==table2["nombre2"],how="left").display()`
* RIGHT (ou appelé communément en SQL `RIGHT OUTER JOIN`)
  - `table1.join(table2, on=table1["nombre1"]==table2["nombre2"],how="right").display()`
  
> Vous pouvez à la place de "left" ou "right" écrire "left_outer" ou "right_outer"

Jointure **LEFT**

In [0]:
# Nous récuperons nos dataframe
mon_dataframe_A = spark.table("formation.arrivees")
mon_dataframe_B = spark.table("formation.stockees")

# Affichage du nombre de rows -> 409317
print(mon_dataframe_A.count())

# Nous appliquons la jointure 
mon_dataframe_C = mon_dataframe_A.join(mon_dataframe_B ,on=((mon_dataframe_A["numero_caf"]==mon_dataframe_B["numero_caf"]) & (mon_dataframe_A["date_jour"]==mon_dataframe_B["date_jour"])), how="left")

# Affichage du nombre de rows -> 2094958
print(mon_dataframe_C.count())

# Affichage de notre table
display(mon_dataframe_C)

Jointure **RIGHT**

In [0]:
# Nous récuperons nos dataframe
mon_dataframe_A = spark.table("formation.arrivees")
mon_dataframe_B = spark.table("formation.stockees")

# Affichage du nombre de rows -> 409317
print(mon_dataframe_A.count())

# Nous appliquons la jointure 
mon_dataframe_C = mon_dataframe_A.join(mon_dataframe_B ,on=((mon_dataframe_A["numero_caf"]==mon_dataframe_B["numero_caf"]) & (mon_dataframe_A["date_jour"]==mon_dataframe_B["date_jour"])), how="right")

# Affichage du nombre de rows -> 2095623
print(mon_dataframe_C.count())

# Affichage de notre table
display(mon_dataframe_C)